# 📊 Projektarbeit SP: Was macht ein Spiel erfolgreich?
- **Gruppe**: XX
- **Namen**: Max Mustermann, ...
- **Ziel**: Analyse von Erfolgsfaktoren bei Steam-Spielen auf Basis von Bewertung, Genre, Spielzeit, Altersfreigabe etc.
- **Quelle**: Daten über RAWG API


In [ ]:
# 🔑 API-Verbindung zur RAWG.io API
import requests

API_KEY = 'DEIN_API_KEY_HIER'
BASE_URL = 'https://api.rawg.io/api/games'

params = {
    'key': API_KEY,
    'page_size': 40,
    'dates': '2015-01-01,2025-12-31'
}
response = requests.get(BASE_URL, params=params)
data = response.json()
data['results'][0]

In [ ]:
# 🧹 Datenaufbereitung
import pandas as pd

games_data = []
for game in data['results']:
    if not game.get('rating') or not game.get('released'):
        continue
    games_data.append({
        'name': game['name'],
        'rating': game['rating'],
        'ratings_count': game['ratings_count'],
        'released': game['released'],
        'genres': [g['name'] for g in game['genres']],
        'esrb_rating': game['esrb_rating']['name'] if game['esrb_rating'] else None
    })

df = pd.DataFrame(games_data)
df['year'] = pd.to_datetime(df['released']).dt.year
df.head()

In [ ]:
# 💾 Speichern in SQLite und SQL-Abfrage
import sqlite3

conn = sqlite3.connect('games.db')
df.to_sql('games', conn, if_exists='replace', index=False)

pd.read_sql("SELECT name, rating FROM games WHERE rating > 4.5 LIMIT 10", conn)

In [ ]:
# 📈 Visualisierung nach ESRB Rating
import matplotlib.pyplot as plt

df.groupby('esrb_rating')['rating'].mean().plot(kind='bar', title='Durchschnittliche Bewertung nach Altersfreigabe')
plt.ylabel('Bewertung (0-5)')
plt.show()

In [ ]:
# 📊 Statistische Analyse: Bewertung vs. Anzahl Reviews
from scipy.stats import pearsonr

x = df['ratings_count']
y = df['rating']
corr, p_value = pearsonr(x, y)
print(f"Korrelationskoeffizient: {corr:.2f}, p-Wert: {p_value:.4f}")

# 📌 Fazit & Diskussion
- Was sind eure Erkenntnisse?
- Was sind mögliche Einschränkungen?
- Was würdet ihr in einer zweiten Version verbessern?
